In [1]:
import numpy as np
import pandas as pd
import datetime
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
import os
import warnings
warnings.filterwarnings('ignore')

In [62]:
path="D:\Python\Elo"
def add_hist_data():
    a=pd.read_csv(os.path.join(path,'historical_transactions.csv'))
    a['category_2'].fillna(1.0,inplace=True)
    a['category_3'].fillna('A',inplace=True)
    a['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True)
    a['installments'].replace(-1, np.nan,inplace=True)
    a['installments'].replace(999, np.nan,inplace=True)
    a['authorized_flag'] = a['authorized_flag'].map({'Y': 1, 'N': 0}).astype(int)
    aggs={
        'purchase_amount':['mean','sum','count','std'],
    'merchant_category_id':['nunique'],
    'category_2':['nunique'],
   'subsector_id':['nunique'],
    'authorized_flag':['mean'],
    'installments':['mean','sum']}
    a=a.pivot_table(index='card_id',columns='month_lag',aggfunc=aggs)
    col=['hist_'+s1+'_'+s2+'_'+str(s3) for (s1,s2,s3) in a.columns.tolist()]
    a.columns=col
    a=a.fillna(0)
    a['hist_purchase_sum_last_ratio']=a['hist_purchase_amount_sum_0']/a['hist_purchase_amount_sum_-1']
    a['hist_purchase_count_last_ratio']=a['hist_purchase_amount_count_0']/a['hist_purchase_amount_count_-1']
    a['hist_purchase_mean_last_ratio']=a['hist_purchase_amount_mean_0']/a['hist_purchase_amount_mean_-1']
    a['hist_purchase_std_last_ratio']=a['hist_purchase_amount_std_0']/a['hist_purchase_amount_std_-1']

    a['hist_purchase_sum_last_3month_mean']=(a['hist_purchase_amount_sum_0']+a['hist_purchase_amount_sum_-1']+a['hist_purchase_amount_sum_-2'])/3
    a['hist_purchase_count_last_3month_mean']=(a['hist_purchase_amount_count_0']+a['hist_purchase_amount_count_-1']+a['hist_purchase_amount_count_-2'])/3
    a['hist_purchase_mean_last_3month_mean']=(a['hist_purchase_amount_mean_0']+a['hist_purchase_amount_mean_-1']+a['hist_purchase_amount_mean_-2'])/3
    a['hist_purchase_std_last_3month_mean']=(a['hist_purchase_amount_std_0']+a['hist_purchase_amount_std_-1']+a['hist_purchase_amount_std_-2'])/3

    a['hist_purchase_sum_last_3month_sum']=(a['hist_purchase_amount_sum_0']+a['hist_purchase_amount_sum_-1']+a['hist_purchase_amount_sum_-2'])/1
    a['hist_purchase_count_last_3month_sum']=(a['hist_purchase_amount_count_0']+a['hist_purchase_amount_count_-1']+a['hist_purchase_amount_count_-2'])/1
    a['hist_purchase_mean_last_3month_sum']=(a['hist_purchase_amount_mean_0']+a['hist_purchase_amount_mean_-1']+a['hist_purchase_amount_mean_-2'])/1
    a['hist_purchase_std_last_3month_sum']=(a['hist_purchase_amount_std_0']+a['hist_purchase_amount_std_-1']+a['hist_purchase_amount_std_-2'])/1

    a['hist_purchase_sum_last_ratio_6month']=a['hist_purchase_amount_sum_0']/a['hist_purchase_amount_sum_-5']
    a['hist_purchase_count_last_ratio_6month']=a['hist_purchase_amount_count_0']/a['hist_purchase_amount_count_-5']
    a['hist_purchase_mean_last_ratio_6month']=a['hist_purchase_amount_mean_0']/a['hist_purchase_amount_mean_-5']
    a['hist_purchase_std_last_ratio_6month']=a['hist_purchase_amount_std_0']/a['hist_purchase_amount_std_-5']

    a['hist_purchase_sum_last_ratio_12month']=a['hist_purchase_amount_sum_0']/a['hist_purchase_amount_sum_-12']
    a['hist_purchase_count_last_ratio_12month']=a['hist_purchase_amount_count_0']/a['hist_purchase_amount_count_-12']
    a['hist_purchase_mean_last_ratio_12month']=a['hist_purchase_amount_mean_0']/a['hist_purchase_amount_mean_-12']
    a['hist_purchase_std_last_ratio_12month']=a['hist_purchase_amount_std_0']/a['hist_purchase_amount_std_-12']

    a['hist_purchase_sum_last_ratio_mean_1month']=a['hist_purchase_amount_sum_0']/((a['hist_purchase_amount_sum_-1']+a['hist_purchase_amount_sum_-2']+a['hist_purchase_amount_sum_-3']+a['hist_purchase_amount_sum_-4']+a['hist_purchase_amount_sum_-5']+a['hist_purchase_amount_sum_-6']+a['hist_purchase_amount_sum_-7'])/7)
    a['hist_purchase_count_last_ratio_mean_1month']=a['hist_purchase_amount_count_0']/((a['hist_purchase_amount_count_-1']+a['hist_purchase_amount_count_-2']+a['hist_purchase_amount_count_-3']+a['hist_purchase_amount_count_-4']+a['hist_purchase_amount_count_-5']+a['hist_purchase_amount_count_-6']+a['hist_purchase_amount_count_-7'])/7)
    a['hist_purchase_mean_last_ratio_mean_1month']=a['hist_purchase_amount_mean_0']/((a['hist_purchase_amount_mean_-1']+a['hist_purchase_amount_mean_-2']+a['hist_purchase_amount_mean_-3']+a['hist_purchase_amount_mean_-4']+a['hist_purchase_amount_mean_-5']+a['hist_purchase_amount_mean_-6']+a['hist_purchase_amount_mean_-7'])/7)
    a['hist_purchase_std_last_ratio_mean_1month']=a['hist_purchase_amount_std_0']/((a['hist_purchase_amount_std_-1']+a['hist_purchase_amount_std_-2']+a['hist_purchase_amount_std_-3']+a['hist_purchase_amount_std_-4']+a['hist_purchase_amount_std_-5']+a['hist_purchase_amount_std_-6']+a['hist_purchase_amount_std_-7'])/7)

    a['hist_purchase_sum_last_ratio_mean_3month']=(a['hist_purchase_amount_sum_0']+a['hist_purchase_amount_sum_-1']+a['hist_purchase_amount_sum_-2'])/(a['hist_purchase_amount_sum_-3']+a['hist_purchase_amount_sum_-4']+a['hist_purchase_amount_sum_-5'])
    a['hist_purchase_count_last_ratio_mean_3month']=(a['hist_purchase_amount_count_0']+a['hist_purchase_amount_count_-1']+a['hist_purchase_amount_count_-2'])/(a['hist_purchase_amount_count_-3']+a['hist_purchase_amount_count_-4']+a['hist_purchase_amount_count_-5'])
    a['hist_purchase_mean_last_ratio_mean_3month']=(a['hist_purchase_amount_mean_0']+a['hist_purchase_amount_mean_-1']+a['hist_purchase_amount_mean_-2'])/(a['hist_purchase_amount_mean_-3']+a['hist_purchase_amount_mean_-4']+a['hist_purchase_amount_mean_-5'])
    a['hist_purchase_std_last_ratio_mean_3month']=(a['hist_purchase_amount_std_0']+a['hist_purchase_amount_std_-1']+a['hist_purchase_amount_std_-2'])/(a['hist_purchase_amount_std_-3']+a['hist_purchase_amount_std_-4']+a['hist_purchase_amount_std_-5'])

    a['hist_purchase_sum_last_many_ratio']=((a['hist_purchase_amount_sum_0']/a['hist_purchase_amount_sum_-1'])+(a['hist_purchase_amount_sum_-1']/a['hist_purchase_amount_sum_-2'])+(a['hist_purchase_amount_sum_-2']/a['hist_purchase_amount_sum_-3'])+(a['hist_purchase_amount_sum_-3']/a['hist_purchase_amount_sum_-4']))/4
    a['hist_purchase_mean_last_many_ratio']=((a['hist_purchase_amount_mean_0']/a['hist_purchase_amount_mean_-1'])+(a['hist_purchase_amount_mean_-1']/a['hist_purchase_amount_mean_-2'])+(a['hist_purchase_amount_mean_-2']/a['hist_purchase_amount_mean_-3'])+(a['hist_purchase_amount_mean_-3']/a['hist_purchase_amount_mean_-4']))/4
    return a
hist=add_hist_data()
hist.to_csv(os.path.join(path,'hard_data_hist.csv'))

In [63]:
def add_new_data():
    a=pd.read_csv(os.path.join(path,'new_merchant_transactions.csv'))
    a['category_2'].fillna(1.0,inplace=True)
    a['category_3'].fillna('A',inplace=True)
    a['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True)
    a['installments'].replace(-1, np.nan,inplace=True)
    a['installments'].replace(999, np.nan,inplace=True)
    a['authorized_flag'] = a['authorized_flag'].map({'Y': 1, 'N': 0}).astype(int)
    aggs={
        'purchase_amount':['mean','sum','count','std'],
    'merchant_category_id':['nunique'],
    'category_2':['nunique'],
   'subsector_id':['nunique'],
    'authorized_flag':['mean'],
    'installments':['mean','sum']}
    a=a.pivot_table(index='card_id',columns='month_lag',aggfunc=aggs)
    col=['new_'+s1+'_'+s2+'_'+str(s3) for (s1,s2,s3) in a.columns.tolist()]
    a.columns=col
    a=a.fillna(0)
    a['new_purchase_sum_last_ratio']=a['new_purchase_amount_sum_1']/a['new_purchase_amount_sum_2']
    a['new_purchase_count_last_ratio']=a['new_purchase_amount_count_1']/a['new_purchase_amount_count_2']
    a['new_purchase_mean_last_ratio']=a['new_purchase_amount_mean_1']/a['new_purchase_amount_mean_2']
    a['new_purchase_std_last_ratio']=a['new_purchase_amount_std_1']/a['new_purchase_amount_std_2']

    a['new_purchase_sum_last_3month_mean']=(a['new_purchase_amount_sum_2']+a['new_purchase_amount_sum_1'])/2
    a['new_purchase_count_last_3month_mean']=(a['new_purchase_amount_count_2']+a['new_purchase_amount_count_1'])/2
    a['new_purchase_mean_last_3month_mean']=(a['new_purchase_amount_mean_2']+a['new_purchase_amount_mean_1'])/2
    a['new_purchase_std_last_3month_mean']=(a['new_purchase_amount_std_2']+a['new_purchase_amount_std_1'])/2

    a['new_purchase_sum_last_3month_sum']=(a['new_purchase_amount_sum_2']+a['new_purchase_amount_sum_1'])/1
    a['new_purchase_count_last_3month_sum']=(a['new_purchase_amount_count_2']+a['new_purchase_amount_count_1'])/1
    a['new_purchase_mean_last_3month_sum']=(a['new_purchase_amount_mean_2']+a['new_purchase_amount_mean_1'])/1
    a['new_purchase_std_last_3month_sum']=(a['new_purchase_amount_std_2']+a['new_purchase_amount_std_1'])/1
    return a
new=add_new_data()
new.to_csv(os.path.join(path,'hard_data_new.csv'))

In [2]:
#Last hist data
path="D:\Python\Elo"
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df
def add_hist_data():
    def subsector_data():
        a=pd.read_csv(os.path.join(path,'historical_transactions.csv'))
        a=reduce_mem_usage(a)
        a['category_2'].fillna(1.0,inplace=True)
        a['category_3'].fillna('A',inplace=True)
        a['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True)
        a['installments'].replace(-1, np.nan,inplace=True)
        a['installments'].replace(999, np.nan,inplace=True)
        a['authorized_flag'] = a['authorized_flag'].map({'Y': 1, 'N': 0}).astype(int)
        aggs={
            'purchase_amount':['mean','sum','count'],
        'merchant_category_id':['nunique'],
        'authorized_flag':['mean'],
        'installments':['mean','sum']}
        a=a.pivot_table(index='card_id',columns='subsector_id',aggfunc=aggs)
        col=['hist_last_'+s1+'_'+s2+'_'+str(s3) for (s1,s2,s3) in a.columns.tolist()]
        a.columns=col
        a=a.fillna(0)
        data=pd.read_csv(os.path.join(path,'train.csv'))
        a=a.merge(data[['card_id','target']],how='left',on='card_id')
        del data;gc.collect()
    #    cor_col=list(a.corr()['target'][a.corr()['target'].abs()>0.02].index)
        cr=a.corr()['target']
        cor_col=list(cr[cr.abs()>0.02].index)
        cor_col.append('card_id')
        a=a[cor_col]
        a=a.drop(['target'],axis=1)
        return a
    def flag_data():
        a=pd.read_csv(os.path.join(path,'historical_transactions.csv'))
        a=reduce_mem_usage(a)
        a['category_2'].fillna(1.0,inplace=True)
        a['category_3'].fillna('A',inplace=True)
        a['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True)
        a['installments'].replace(-1, np.nan,inplace=True)
        a['installments'].replace(999, np.nan,inplace=True)
        a['authorized_flag'] = a['authorized_flag'].map({'Y': 1, 'N': 0}).astype(int)
        aggs={
            'purchase_amount':['mean','sum','count'],
        'merchant_category_id':['nunique'],
        'subsector_id':['nunique'],
        'installments':['mean','sum']}
        a=a.pivot_table(index='card_id',columns='authorized_flag',aggfunc=aggs)
        col=['hist_last_'+s1+'_'+s2+'_'+str(s3) for (s1,s2,s3) in a.columns.tolist()]
        a.columns=col
        a=a.fillna(0)
        data=pd.read_csv(os.path.join(path,'train.csv'))
        a=a.merge(data[['card_id','target']],how='left',on='card_id')
        del data;gc.collect()
    #    cor_col=list(a.corr()['target'][a.corr()['target'].abs()>0.02].index)
        cr=a.corr()['target']
        cor_col=list(cr[cr.abs()>0.02].index)
        cor_col.append('card_id')
        a=a[cor_col]
        a=a.drop(['target'],axis=1)
        return a
    def cat2_data():
        a=pd.read_csv(os.path.join(path,'historical_transactions.csv'))
        a=reduce_mem_usage(a)
        a['category_2'].fillna(1.0,inplace=True)
        a['category_3'].fillna('A',inplace=True)
        a['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True)
        a['installments'].replace(-1, np.nan,inplace=True)
        a['installments'].replace(999, np.nan,inplace=True)
        a['authorized_flag'] = a['authorized_flag'].map({'Y': 1, 'N': 0}).astype(int)
        aggs={
            'purchase_amount':['mean','sum','count'],
        'authorized_flag':['mean'],
        'subsector_id':['nunique'],
        'installments':['mean','sum']}
        a=a.pivot_table(index='card_id',columns='category_2',aggfunc=aggs)
        col=['hist_last_'+s1+'_'+s2+'_'+str(s3) for (s1,s2,s3) in a.columns.tolist()]
        a.columns=col
        a=a.fillna(0)
        data=pd.read_csv(os.path.join(path,'train.csv'))
        a=a.merge(data[['card_id','target']],how='left',on='card_id')
        del data;gc.collect()
    #    cor_col=list(a.corr()['target'][a.corr()['target'].abs()>0.02].index)
        cr=a.corr()['target']
        cor_col=list(cr[cr.abs()>0.02].index)
        cor_col.append('card_id')
        a=a[cor_col]
        a=a.drop(['target'],axis=1)
        return a
    def cat3_data():
        a=pd.read_csv(os.path.join(path,'historical_transactions.csv'))
        a=reduce_mem_usage(a)
        a['category_2'].fillna(1.0,inplace=True)
        a['category_3'].fillna('A',inplace=True)
        a['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True)
        a['installments'].replace(-1, np.nan,inplace=True)
        a['installments'].replace(999, np.nan,inplace=True)
        a['authorized_flag'] = a['authorized_flag'].map({'Y': 1, 'N': 0}).astype(int)
        aggs={
            'purchase_amount':['mean','sum','count'],
        'authorized_flag':['mean'],
        'subsector_id':['nunique'],
        'installments':['mean','sum']}
        a=a.pivot_table(index='card_id',columns='category_3',aggfunc=aggs)
        col=['hist_last_'+s1+'_'+s2+'_'+str(s3) for (s1,s2,s3) in a.columns.tolist()]
        a.columns=col
        a=a.fillna(0)
        data=pd.read_csv(os.path.join(path,'train.csv'))
        a=a.merge(data[['card_id','target']],how='left',on='card_id')
        del data;gc.collect()
    #    cor_col=list(a.corr()['target'][a.corr()['target'].abs()>0.02].index)
        cr=a.corr()['target']
        cor_col=list(cr[cr.abs()>0.02].index)
        cor_col.append('card_id')
        a=a[cor_col]
        a=a.drop(['target'],axis=1)
        return a
    def one_month_data():
        df = pd.read_csv(os.path.join(path,'historical_transactions.csv'))
        df=reduce_mem_usage(df)
        df['category_2'].fillna(1.0,inplace=True)
        df['category_3'].fillna('A',inplace=True)
        df['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True)
        df['installments'].replace(-1, np.nan,inplace=True)
        df['installments'].replace(999, np.nan,inplace=True)
        df=df[df['month_lag']==0].reset_index(drop=True)
        for col in ['category_2','category_3']:
            df[col+'_mean'] = df.groupby([col])['purchase_amount'].transform('mean')
            df[col+'_min'] = df.groupby([col])['purchase_amount'].transform('min')
            df[col+'_max'] = df.groupby([col])['purchase_amount'].transform('max')
            df[col+'_sum'] = df.groupby([col])['purchase_amount'].transform('sum')
        df['purchase_date']=pd.to_datetime(df['purchase_date'])  
        df['day'] = df['purchase_date'].dt.day
        df['weekday'] = df['purchase_date'].dt.dayofweek
        df['weekend'] = (df.purchase_date.dt.weekday >=5).astype(np.int32)
        df['hour'] = df['purchase_date'].dt.hour
        df['authorized_flag'] = df['authorized_flag'].map({'Y':1, 'N':0})
        df['category_1'] = df['category_1'].map({'Y':1, 'N':0}) 
        df['price'] = df['purchase_amount'] / df['installments']
        df=reduce_mem_usage(df)
        def get_new_columns(name,aggs):
            return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]
        aggs={
        'merchant_id': ['nunique'],
        'day':['nunique','mean','min'],
        'weekday':['mean'],
        'weekend':['mean'],
        'hour':['mean','nunique','min','max'],
        'merchant_category_id': ['nunique'],
        'state_id': ['nunique'],
        'city_id': ['nunique'],
        'authorized_flag':['nunique']  ,  
        'subsector_id': ['nunique'],
        'purchase_amount': ['sum', 'mean', 'max', 'min', 'std'],
        'installments': ['sum', 'mean', 'max', 'min', 'std'],
        'category_1':['sum', 'mean'],
        'category_2_mean': ['mean'],
        'category_3_mean': ['mean'],
        'category_2_min': ['mean'],
        'category_3_min': ['mean'],
        'category_2_max': ['mean'],
        'category_3_max': ['mean'],
        'category_2_sum': ['mean'],
        'category_3_sum': ['mean'],
        'card_id':['size','count'],
        'price':['mean','min','max','var']
        }
        new_columns=get_new_columns('hist_month',aggs)
        df=df.groupby(['card_id']).agg(aggs)
        df.columns=new_columns
        df.reset_index(drop=False,inplace=True)
        return df
    def two_month_data():
        df = pd.read_csv(os.path.join(path,'historical_transactions.csv'))
        df=reduce_mem_usage(df)
        df['category_2'].fillna(1.0,inplace=True)
        df['category_3'].fillna('A',inplace=True)
        df['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True)
        df['installments'].replace(-1, np.nan,inplace=True)
        df['installments'].replace(999, np.nan,inplace=True)
        df=df[df['month_lag']==-1].reset_index(drop=True)
        for col in ['category_2','category_3']:
            df[col+'_mean'] = df.groupby([col])['purchase_amount'].transform('mean')
            df[col+'_min'] = df.groupby([col])['purchase_amount'].transform('min')
            df[col+'_max'] = df.groupby([col])['purchase_amount'].transform('max')
            df[col+'_sum'] = df.groupby([col])['purchase_amount'].transform('sum')
        df['purchase_date']=pd.to_datetime(df['purchase_date'])  
        df['day'] = df['purchase_date'].dt.day
        df['weekday'] = df['purchase_date'].dt.dayofweek
        df['weekend'] = (df.purchase_date.dt.weekday >=5).astype(np.int32)
        df['hour'] = df['purchase_date'].dt.hour
        df['authorized_flag'] = df['authorized_flag'].map({'Y':1, 'N':0})
        df['category_1'] = df['category_1'].map({'Y':1, 'N':0}) 
        df['price'] = df['purchase_amount'] / df['installments']
        df=reduce_mem_usage(df)
        def get_new_columns(name,aggs):
            return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]
        aggs={
        'merchant_id': ['nunique'],
        'day':['nunique','mean','min'],
        'weekday':['mean'],
        'weekend':['mean'],
        'hour':['mean','nunique','min','max'],
        'merchant_category_id': ['nunique'],
        'state_id': ['nunique'],
        'city_id': ['nunique'],
        'authorized_flag':['nunique']  ,  
        'subsector_id': ['nunique'],
        'purchase_amount': ['sum', 'mean', 'max', 'min', 'std'],
        'installments': ['sum', 'mean', 'max', 'min', 'std'],
        'category_1':['sum', 'mean'],
        'category_2_mean': ['mean'],
        'category_3_mean': ['mean'],
        'category_2_min': ['mean'],
        'category_3_min': ['mean'],
        'category_2_max': ['mean'],
        'category_3_max': ['mean'],
        'category_2_sum': ['mean'],
        'category_3_sum': ['mean'],
        'card_id':['size','count'],
        'price':['mean','min','max','var']
        }
        new_columns=get_new_columns('hist_two_month',aggs)
        df=df.groupby(['card_id']).agg(aggs)
        df.columns=new_columns
        df.reset_index(drop=False,inplace=True)
        return df
    def ratio_month(df):
        cl_one=[a for a in df.columns if 'hist_month' in str(a)]
        cl_two=[a for a in df.columns if 'hist_two_month' in str(a)]
        for one,two in zip(cl_one,cl_two):
            df[one+'_ratio']=df[one]/df[two]
        df=df.drop(cl_two,axis=1)
        return df
    data=subsector_data()
    temp_data=one_month_data()
    data=data.merge(temp_data,how='left',on='card_id')
#    temp_data=two_month_data()
#    data=data.merge(temp_data,how='left',on='card_id')
    temp_data=flag_data()
    data=data.merge(temp_data,how='left',on='card_id')
    temp_data=cat2_data()
    data=data.merge(temp_data,how='left',on='card_id')
    temp_data=cat3_data()
    data=data.merge(temp_data,how='left',on='card_id')
#    data=ratio_month(data)
    del temp_data;gc.collect();
    return data
hist=add_hist_data()
hist.to_csv(os.path.join(path,'hist_last_update.csv'))
del hist;gc.collect();

Mem. usage decreased to 1749.11 Mb (43.7% reduction)
Mem. usage decreased to 1749.11 Mb (43.7% reduction)
Mem. usage decreased to 281.36 Mb (38.0% reduction)
Mem. usage decreased to 1749.11 Mb (43.7% reduction)
Mem. usage decreased to 1749.11 Mb (43.7% reduction)
Mem. usage decreased to 1749.11 Mb (43.7% reduction)


In [3]:
#Last new data
def add_new_data():
    def subsector_data():
        a=pd.read_csv(os.path.join(path,'new_merchant_transactions.csv'))
        a['category_2'].fillna(1.0,inplace=True)
        a['category_3'].fillna('A',inplace=True)
        a['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True)
        a['installments'].replace(-1, np.nan,inplace=True)
        a['installments'].replace(999, np.nan,inplace=True)
        a['authorized_flag'] = a['authorized_flag'].map({'Y': 1, 'N': 0}).astype(int)
        aggs={
            'purchase_amount':['mean','sum','count'],
        'merchant_category_id':['nunique'],
        'authorized_flag':['mean'],
        'installments':['mean','sum']}
        a=a.pivot_table(index='card_id',columns='subsector_id',aggfunc=aggs)
        col=['new_last_'+s1+'_'+s2+'_'+str(s3) for (s1,s2,s3) in a.columns.tolist()]
        a.columns=col
        a=a.fillna(0)
        data=pd.read_csv(os.path.join(path,'train.csv'))
        a=a.merge(data[['card_id','target']],how='left',on='card_id')
        del data;gc.collect()
    #    cor_col=list(a.corr()['target'][a.corr()['target'].abs()>0.02].index)
        cr=a.corr()['target']
        cor_col=list(cr[cr.abs()>0.02].index)
        cor_col.append('card_id')
        a=a[cor_col]
        a=a.drop(['target'],axis=1)
        a=reduce_mem_usage(a)
        return a
    def cat2_data():
        a=pd.read_csv(os.path.join(path,'new_merchant_transactions.csv'))
        a['category_2'].fillna(1.0,inplace=True)
        a['category_3'].fillna('A',inplace=True)
        a['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True)
        a['installments'].replace(-1, np.nan,inplace=True)
        a['installments'].replace(999, np.nan,inplace=True)
        a['authorized_flag'] = a['authorized_flag'].map({'Y': 1, 'N': 0}).astype(int)
        aggs={
            'purchase_amount':['mean','sum','count'],
        'authorized_flag':['mean'],
        'subsector_id':['nunique'],
        'installments':['mean','sum']}
        a=a.pivot_table(index='card_id',columns='category_2',aggfunc=aggs)
        col=['new_last_'+s1+'_'+s2+'_'+str(s3) for (s1,s2,s3) in a.columns.tolist()]
        a.columns=col
        a=a.fillna(0)
        data=pd.read_csv(os.path.join(path,'train.csv'))
        a=a.merge(data[['card_id','target']],how='left',on='card_id')
        del data;gc.collect()
    #    cor_col=list(a.corr()['target'][a.corr()['target'].abs()>0.02].index)
        cr=a.corr()['target']
        cor_col=list(cr[cr.abs()>0.02].index)
        cor_col.append('card_id')
        a=a[cor_col]
        a=a.drop(['target'],axis=1)
        a=reduce_mem_usage(a)
        return a
    def cat3_data():
        a=pd.read_csv(os.path.join(path,'new_merchant_transactions.csv'))
        a['category_2'].fillna(1.0,inplace=True)
        a['category_3'].fillna('A',inplace=True)
        a['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True)
        a['installments'].replace(-1, np.nan,inplace=True)
        a['installments'].replace(999, np.nan,inplace=True)
        a['authorized_flag'] = a['authorized_flag'].map({'Y': 1, 'N': 0}).astype(int)
        aggs={
            'purchase_amount':['mean','sum','count'],
        'authorized_flag':['mean'],
        'subsector_id':['nunique'],
        'installments':['mean','sum']}
        a=a.pivot_table(index='card_id',columns='category_3',aggfunc=aggs)
        col=['new_last_'+s1+'_'+s2+'_'+str(s3) for (s1,s2,s3) in a.columns.tolist()]
        a.columns=col
        a=a.fillna(0)
        data=pd.read_csv(os.path.join(path,'train.csv'))
        a=a.merge(data[['card_id','target']],how='left',on='card_id')
        del data;gc.collect()
    #    cor_col=list(a.corr()['target'][a.corr()['target'].abs()>0.02].index)
        cr=a.corr()['target']
        cor_col=list(cr[cr.abs()>0.02].index)
        cor_col.append('card_id')
        a=a[cor_col]
        a=a.drop(['target'],axis=1)
        a=reduce_mem_usage(a)
        return a
    def one_month_data():
        df = pd.read_csv(os.path.join(path,'new_merchant_transactions.csv'))
        df=reduce_mem_usage(df)
        df['category_2'].fillna(1.0,inplace=True)
        df['category_3'].fillna('A',inplace=True)
        df['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True)
        df['installments'].replace(-1, np.nan,inplace=True)
        df['installments'].replace(999, np.nan,inplace=True)
        df=df[df['month_lag']==2].reset_index(drop=True)
        for col in ['category_2','category_3']:
            df[col+'_mean'] = df.groupby([col])['purchase_amount'].transform('mean')
            df[col+'_min'] = df.groupby([col])['purchase_amount'].transform('min')
            df[col+'_max'] = df.groupby([col])['purchase_amount'].transform('max')
            df[col+'_sum'] = df.groupby([col])['purchase_amount'].transform('sum')
        df['purchase_date']=pd.to_datetime(df['purchase_date'])      
        df['day'] = df['purchase_date'].dt.day
        df['weekday'] = df['purchase_date'].dt.dayofweek
        df['weekend'] = (df.purchase_date.dt.weekday >=5).astype(np.int32)
        df['hour'] = df['purchase_date'].dt.hour
        df['authorized_flag'] = df['authorized_flag'].map({'Y':1, 'N':0})
        df['category_1'] = df['category_1'].map({'Y':1, 'N':0}) 
        df['price'] = df['purchase_amount'] / df['installments']
        df=reduce_mem_usage(df)
        def get_new_columns(name,aggs):
            return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]
        aggs={
        'merchant_id': ['nunique'],
        'day':['nunique','mean','min'],
        'weekday':['mean'],
        'weekend':['mean'],
        'hour':['mean','nunique','min','max'],
        'merchant_category_id': ['nunique'],
        'state_id': ['nunique'],
        'city_id': ['nunique'],
        'authorized_flag':['nunique']  ,  
        'subsector_id': ['nunique'],
        'purchase_amount': ['sum', 'mean', 'max', 'min', 'std'],
        'installments': ['sum', 'mean', 'max', 'min', 'std'],
        'category_1':['sum', 'mean'],
        'category_2_mean': ['mean'],
        'category_3_mean': ['mean'],
        'category_2_min': ['mean'],
        'category_3_min': ['mean'],
        'category_2_max': ['mean'],
        'category_3_max': ['mean'],
        'category_2_sum': ['mean'],
        'category_3_sum': ['mean'],
        'card_id':['size','count'],
        'price':['mean','min','max','var']
        }
        new_columns=get_new_columns('new_month',aggs)
        df=df.groupby(['card_id']).agg(aggs)
        df.columns=new_columns
        df.reset_index(drop=False,inplace=True)
        return df
    data=subsector_data()
    temp_data=one_month_data()
    data=data.merge(temp_data,how='left',on='card_id')
    temp_data=cat2_data()
    data=data.merge(temp_data,how='left',on='card_id')
    temp_data=cat3_data()
    data=data.merge(temp_data,how='left',on='card_id')
    del temp_data;gc.collect();
    return data
new=add_new_data()
new.to_csv(os.path.join(path,'new_last_update.csv'))

Mem. usage decreased to 23.78 Mb (70.9% reduction)
Mem. usage decreased to 114.20 Mb (45.5% reduction)
Mem. usage decreased to 77.61 Mb (26.9% reduction)
Mem. usage decreased to  8.30 Mb (58.3% reduction)
Mem. usage decreased to  9.96 Mb (62.5% reduction)


In [10]:
cl_one=[a for a in hist.columns if 'hist_month' in str(a)]
cl_two=[a for a in hist.columns if 'hist_two_month' in str(a)]
for one,two in zip(cl_one,cl_two):
    hist[one+'_ratio']=hist[one]/hist[two]

In [15]:
hist

,hist_last_authorized_flag_mean_8,hist_last_installments_mean_7,hist_last_installments_mean_8,hist_last_installments_mean_19,hist_last_installments_mean_34,hist_last_installments_mean_37,hist_last_installments_sum_7,hist_last_installments_sum_8,hist_last_installments_sum_22,hist_last_installments_sum_29,...,hist_last_installments_sum_1.0,hist_last_authorized_flag_mean_A,hist_last_installments_mean_B,hist_last_installments_sum_B,hist_last_installments_sum_C,hist_last_purchase_amount_count_B,hist_last_purchase_amount_count_C,hist_last_subsector_id_nunique_B,hist_last_subsector_id_nunique_C,card_id
0,0.857143,0.000000,1.0,1.020833,1.000,0.000000,0.0,28.0,0.0,1.0,...,28.0,0.000000,1.0,125.0,67.0,125.0,24.0,12.0,7.0,C_ID_00007093c1
1,0.000000,1.000000,0.0,1.000000,1.125,1.000000,2.0,0.0,0.0,11.0,...,170.0,1.000000,1.0,88.0,113.0,88.0,32.0,13.0,8.0,C_ID_0001238066
2,0.666667,0.000000,0.0,0.000000,0.000,0.000000,0.0,0.0,0.0,1.0,...,0.0,0.953846,1.0,1.0,0.0,1.0,0.0,1.0,0.0,C_ID_0001506ef0
3,0.000000,0.000000,1.0,0.000000,0.000,0.020833,0.0,2.0,0.0,1.0,...,5.0,0.895735,1.0,5.0,0.0,5.0,0.0,4.0,0.0,C_ID_0001793786
4,0.000000,1.000000,0.0,1.000000,1.000,1.000000,3.0,0.0,0.0,10.0,...,33.0,1.000000,1.0,103.0,165.0,103.0,37.0,16.0,14.0,C_ID_000183fdda
5,0.000000,0.000000,0.0,0.000000,0.000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.815385,1.0,5.0,0.0,5.0,0.0,1.0,0.0,C_ID_00024e244b
6,1.000000,1.000000,1.0,1.000000,0.000,1.000000,4.0,3.0,0.0,30.0,...,37.0,1.000000,1.0,59.0,71.0,59.0,13.0,11.0,4.0,C_ID_0002709b5a
7,0.000000,0.000000,0.0,0.000000,0.000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.571429,0.0,0.0,0.0,0.0,0.0,0.0,0.0,C_ID_00027503e2
8,0.000000,0.000000,0.0,0.000000,0.000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.933333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,C_ID_000298032a
9,1.000000,0.000000,0.0,0.000000,0.000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.785714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,C_ID_0002ba3c2e
